In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.assignment2.simulation import HospitalSimulation, PatientState, WardType, WardConfig, WardsConfigurations, HospitalConfiguration
from scipy import stats
import numpy as np
import seaborn as sns


In [3]:

configs = [
    WardConfig(bed_capacity=55, urgency=7, mean_arrival_time=1/14.5, 
        mean_stay_time=2.9),
    WardConfig(40, 5, 1/11, 4),
    WardConfig(30, 2, 1/8, 4.5),
    WardConfig(20, 10, 1/6.5, 1.4),
    WardConfig(20, 5, 1/5, 3.9),
    WardConfig(0, 0, 1/13, 2.2)
]

configs_nof = configs.copy()
del configs_nof[-1]

std_configs = WardsConfigurations(configs)
std_switch = {
    WardType.A: [0.0, 0.05, 0.10, 0.05, 0.80, 0.0],
    WardType.B: [0.20, 0.0, 0.50, 0.15, 0.15, 0.0],
    WardType.C: [0.30, 0.20, 0.0, 0.20, 0.30, 0.0],
    WardType.D: [0.35, 0.30, 0.05, 0.0, 0.30, 0.0],
    WardType.E: [0.20, 0.10, 0.60, 0.10, 0.0, 0.0],
    WardType.F: [0.20, 0.20, 0.20, 0.20, 0.20, 0.0]
}

def mean_exp_dist(mean_time):
    return stats.expon.rvs(scale=mean_time)

sim_config = HospitalConfiguration(std_configs, std_switch, mean_exp_dist, mean_exp_dist)
hsim = HospitalSimulation(sim_config)

In [4]:
result = hsim.sim_multiple_year(100)



In [6]:
occupied_with_F = result.state_from_ward_pct_distribution([PatientState.REJECTED, PatientState.IN_WRONG_WARD], [WardType.C])

number_of_admissions = result.number_of_state_in_ward([PatientState.IN_CORRECT_WARD], [WardType.D])

number_of_relocated = result.number_of_state_from_ward([PatientState.REJECTED, PatientState.IN_WRONG_WARD], [WardType.C])

penalty = result.expected_penalty_from_wards(WardType.A)

([217,
  105,
  28,
  105,
  84,
  154,
  28,
  273,
  28,
  105,
  210,
  28,
  105,
  77,
  266,
  245,
  91,
  56,
  168,
  126,
  91,
  35,
  91,
  98,
  77,
  133,
  70,
  168,
  14,
  49,
  63,
  238,
  161,
  224,
  154,
  175,
  84,
  0,
  77,
  273,
  126,
  252,
  161,
  231,
  98,
  98,
  98,
  84,
  91,
  63,
  63,
  126,
  84,
  98,
  77,
  217,
  28,
  91,
  154,
  189,
  231,
  91,
  77,
  189,
  77,
  322,
  119,
  49,
  35,
  231,
  105,
  203,
  28,
  63,
  63,
  56,
  161,
  112,
  91,
  168,
  91,
  294,
  119,
  35,
  126,
  35,
  63,
  21,
  126,
  147,
  84,
  119,
  49,
  77,
  112,
  119,
  35,
  126,
  28,
  196],
 116.76,
 (106.9694021142467, 126.5505978857533))

In [12]:

test.state_from_ward([PatientState.REJECTED], [WardType.B])

[]

In [5]:
res = hsim.sim_multiple_year(100)

In [8]:
state = [PatientState.REJECTED, PatientState.IN_WRONG_YARD]
res.state_pct_distribution(state, [WardType.A])

/home/simon/anaconda3/envs/StochSim/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2128: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/simon/anaconda3/envs/StochSim/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2129: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


([0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 0.0,
 (nan, nan))

In [6]:
## Example of setting bed distribution, individually and alltogether
hsim.ward_configs[WardType.A].bed_capacity = 40
hsim.ward_configs.bed_distribution = [100, 40, 10, 10, 10, 0]

## Example setting new distributions
def mean_lognorm_sd2(mean_time):
    s = 2/mean_time**2
    mu = mean_time * np.exp(-s**2/2)
    return stats.lognorm(scale=mu, s=np.sqrt(s)).rvs()

hsim.stay_dist = mean_lognorm_sd2


In [10]:
hsim.simulate_year()

In [12]:
hsim.wards[WardType.E].rejected_fraction

0.9308980213089803

In [14]:
[hsim.simulate_year() for _ in range(10)]